In [1]:
import scanpy as sc
import decoupler as dc

In [3]:
import pandas as pd

In [1]:
import warnings
import os
from pathlib import Path 
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import anndata
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import decoupler as dc
import itertools
from scipy import sparse
#import lib.data_helpers as dh

#import lib.scanpy_helpers as sh
#import lib.pl.util as pu

/home/kvalem/.local/lib/python3.11/site-packages/numba/core/cpu.py:99: UserWarning: Numba extension module 'awkward.numba' failed to load due to 'ModuleNotFoundError(No module named 'importlib_metadata')'.
  numba.core.entrypoints.init_all()


In [ ]:
progeny = dc.get_progeny(organism='human', top=500)
progeny

In [52]:
adata.var

,is_highly_variable,mito,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,feature_is_filtered,feature_name,feature_reference,feature_biotype
index,,,,,,,,,,
A1BG,True,False,86866,2.527138,90.681054,2355657.0,False,A1BG,NCBITaxon:9606,gene
A1BG-AS1,True,False,12257,0.369194,98.685074,344142.0,False,A1BG-AS1,NCBITaxon:9606,gene
A2M,True,False,122241,22.579874,86.886039,21047694.0,False,A2M,NCBITaxon:9606,gene
A2M-AS1,False,False,8827,0.172223,99.053043,160537.0,False,A2M-AS1,NCBITaxon:9606,gene
A2ML1,True,False,5096,0.038011,99.453303,35432.0,False,A2ML1,NCBITaxon:9606,gene
...,...,...,...,...,...,...,...,...,...,...
ZXDC,False,False,43291,0.442399,95.355760,412380.0,False,ZXDC,NCBITaxon:9606,gene
ZYG11A,False,False,3517,0.009742,99.622698,9081.0,False,ZYG11A,NCBITaxon:9606,gene
ZYG11B,False,False,71512,0.275276,92.328224,256597.0,False,ZYG11B,NCBITaxon:9606,gene


In [54]:
adata_ex.obs

,n_genes,percent_mito,n_counts,louvain
index,,,,
AAACATACAACCAC-1,781,0.030178,2419.0,CD4 T cells
AAACATTGAGCTAC-1,1352,0.037936,4903.0,B cells
AAACATTGATCAGC-1,1131,0.008897,3147.0,CD4 T cells
AAACCGTGCTTCCG-1,960,0.017431,2639.0,CD14+ Monocytes
AAACCGTGTATGCG-1,522,0.012245,980.0,NK cells
...,...,...,...,...
TTTCGAACTCTCAT-1,1155,0.021104,3459.0,CD14+ Monocytes
TTTCTACTGAGGCA-1,1227,0.009294,3443.0,B cells
TTTCTACTTCCTCG-1,622,0.021971,1684.0,B cells


In [42]:
mat : list, DataFrame or AnnData
    List of [features, matrix], dataframe (samples x features) or an AnnData instance.

AnnData object with n_obs × n_vars = 2638 × 1838
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain'
    var: 'n_cells'
    uns: 'draw_graph', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_draw_graph_fr'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [5]:
path = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/011_analysis_paired_remove_xy/pseudobulk/"
resDir_figures = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/figures/LUAD_DE/"
resDir_tables = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/"


In [6]:
adata = sc.read_h5ad("/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/old/pseudobulk/paired_adata_clean_luad_all_cell_type.h5ad")

In [7]:
# Define the cell types to keep
selected_cell_types = {
    'macrophage',
    'CD4+ T cell',
    'CD8+ T cell',
    'B cell',
    'malignant cell',
    'regulatory T cell',
    'epithelial cell',
    'neutrophil',
    'dendritic cell',
    'plasma cell',
    'endothelial cell',
    'fibroblast'
}

# Mapping for synonymous cell types in adata
cell_type_mapping = {
    'fibroblast of lung': 'fibroblast',
    'vein endothelial cell': 'endothelial cell',
    'capillary endothelial cell': 'endothelial cell',
    'pulmonary artery endothelial cell': 'endothelial cell',
    'endothelial cell of lymphatic vessel': 'endothelial cell',
    'conventional dendritic cell': 'dendritic cell',
    'CD1c-positive myeloid dendritic cell': 'dendritic cell',
    'plasmacytoid dendritic cell': 'dendritic cell',
    'bronchus fibroblast of lung': 'fibroblast'
}

# Standardize cell types based on mapping
adata.obs['standardized_cell_type'] = adata.obs['cell_type'].replace(cell_type_mapping)

# Filter adata to keep only selected cell types
adata = adata[adata.obs['standardized_cell_type'].isin(selected_cell_types)].copy()

# Verify result
print(adata.obs['standardized_cell_type'].value_counts())


/tmp/ipykernel_2966043/291649954.py:31: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  adata.obs['standardized_cell_type'] = adata.obs['cell_type'].replace(cell_type_mapping)


standardized_cell_type
macrophage           104271
CD4+ T cell           94503
CD8+ T cell           72603
epithelial cell       21039
malignant cell        16939
B cell                16853
regulatory T cell     16785
dendritic cell        16702
endothelial cell      16150
plasma cell           15096
neutrophil             6146
fibroblast             4674
Name: count, dtype: int64


## There are malignant cells in normal_adjacent samples
Comming from 59 different patients and 14 datasets
I cannot give an explanation for this so i remove this "mislalbeled" cells 

In [8]:
grouped = adata.obs.groupby(['standardized_cell_type', 'origin']).size().reset_index(name='cell_count')

/tmp/ipykernel_2966043/2956175182.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = adata.obs.groupby(['standardized_cell_type', 'origin']).size().reset_index(name='cell_count')


In [9]:
grouped

,standardized_cell_type,origin,cell_count
0,epithelial cell,normal_adjacent,4957
1,epithelial cell,tumor_primary,16082
2,macrophage,normal_adjacent,65453
3,macrophage,tumor_primary,38818
4,B cell,normal_adjacent,2888
5,B cell,tumor_primary,13965
6,dendritic cell,normal_adjacent,6752
7,dendritic cell,tumor_primary,9950
8,CD4+ T cell,normal_adjacent,45654
9,CD4+ T cell,tumor_primary,48849


In [10]:
# Create a Boolean mask for the cells that you want to keep
mask = ~((adata.obs['origin'] == 'normal_adjacent') & (adata.obs['standardized_cell_type'] == 'malignant cell'))

# Subset the adata object using the mask to exclude the specific cells
adata = adata[mask].copy()

# Now `adata_filtered` contains all cells except those with origin == 'normal_adjacent' and cell_type == 'malignant cell'


In [11]:
grouped = adata.obs.groupby(['standardized_cell_type', 'origin']).size().reset_index(name='cell_count')
grouped

/tmp/ipykernel_2966043/639500846.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = adata.obs.groupby(['standardized_cell_type', 'origin']).size().reset_index(name='cell_count')


,standardized_cell_type,origin,cell_count
0,epithelial cell,normal_adjacent,4957
1,epithelial cell,tumor_primary,16082
2,macrophage,normal_adjacent,65453
3,macrophage,tumor_primary,38818
4,B cell,normal_adjacent,2888
5,B cell,tumor_primary,13965
6,dendritic cell,normal_adjacent,6752
7,dendritic cell,tumor_primary,9950
8,CD4+ T cell,normal_adjacent,45654
9,CD4+ T cell,tumor_primary,48849


In [12]:
adata_normal = adata[adata.obs["origin"]=="normal_adjacent"]
#adata_normal = adata_normal[~adata_normal.obs["cell_type"].isin(["malignant cell"])]

In [13]:
set(adata_normal.obs.cell_type)

{'B cell',
 'CD1c-positive myeloid dendritic cell',
 'CD4+ T cell',
 'CD8+ T cell',
 'bronchus fibroblast of lung',
 'capillary endothelial cell',
 'conventional dendritic cell',
 'dendritic cell',
 'endothelial cell of lymphatic vessel',
 'epithelial cell',
 'fibroblast of lung',
 'macrophage',
 'neutrophil',
 'plasma cell',
 'plasmacytoid dendritic cell',
 'pulmonary artery endothelial cell',
 'regulatory T cell',
 'vein endothelial cell'}

In [14]:
adata_tumor = adata[adata.obs["origin"]=="tumor_primary"]

In [15]:
set(adata_normal.obs.cell_type)

{'B cell',
 'CD1c-positive myeloid dendritic cell',
 'CD4+ T cell',
 'CD8+ T cell',
 'bronchus fibroblast of lung',
 'capillary endothelial cell',
 'conventional dendritic cell',
 'dendritic cell',
 'endothelial cell of lymphatic vessel',
 'epithelial cell',
 'fibroblast of lung',
 'macrophage',
 'neutrophil',
 'plasma cell',
 'plasmacytoid dendritic cell',
 'pulmonary artery endothelial cell',
 'regulatory T cell',
 'vein endothelial cell'}

In [16]:
adata_normal = adata[adata.obs["origin"]=="normal_adjacent"]
#adata_normal = adata_normal[~adata_normal.obs["cell_type"].isin(["malignant cell"])]

In [17]:
adata_tumor = adata[adata.obs["origin"]=="tumor_primary"]

In [18]:
adata

AnnData object with n_obs × n_vars = 401187 × 17811
    obs: 'sample', 'uicc_stage', 'ever_smoker', 'age', 'donor_id', 'origin', 'dataset', 'ann_fine', 'cell_type_predicted', 'doublet_status', 'leiden', 'n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'ann_coarse', 'cell_type_tumor', 'tumor_stage', 'EGFR_mutation', 'TP53_mutation', 'ALK_mutation', 'BRAF_mutation', 'ERBB2_mutation', 'KRAS_mutation', 'ROS_mutation', 'origin_fine', 'study', 'platform', 'cell_type_major', 'cell_type_neutro', 'cell_type_neutro_coarse', 'suspension_type', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'standardized_cell_type'
    var: 'is_highly_variable', 'm

In [19]:
adata_male = adata[adata.obs["sex"]=="male"]

In [20]:
adata_female = adata[adata.obs["sex"]=="female"]

In [56]:
msigdb = dc.get_resource('MSigDB')
msigdb

,genesymbol,collection,geneset
0,MSC,oncogenic_signatures,PKCA_DN.V1_DN
1,MSC,mirna_targets,MIR12123
2,MSC,chemical_and_genetic_perturbations,NIKOLSKY_BREAST_CANCER_8Q12_Q22_AMPLICON
3,MSC,immunologic_signatures,GSE32986_UNSTIM_VS_GMCSF_AND_CURDLAN_LOWDOSE_S...
4,MSC,chemical_and_genetic_perturbations,BENPORATH_PRC2_TARGETS
...,...,...,...
2407729,OR2W5P,immunologic_signatures,GSE22601_DOUBLE_NEGATIVE_VS_CD8_SINGLE_POSITIV...
2407730,OR2W5P,immunologic_signatures,KANNAN_BLOOD_2012_2013_TIV_AGE_65PLS_REVACCINA...
2407731,OR52L2P,immunologic_signatures,GSE22342_CD11C_HIGH_VS_LOW_DECIDUAL_MACROPHAGE...
2407732,CSNK2A3,immunologic_signatures,OCONNOR_PBMC_MENVEO_ACWYVAX_AGE_30_70YO_7DY_AF...


In [57]:
# Filter by hallmark
msigdb = msigdb[msigdb['collection']=='hallmark']

# Remove duplicated entries
msigdb = msigdb[~msigdb.duplicated(['geneset', 'genesymbol'])]
msigdb

,genesymbol,collection,geneset
11,MSC,hallmark,HALLMARK_TNFA_SIGNALING_VIA_NFKB
149,ICOSLG,hallmark,HALLMARK_TNFA_SIGNALING_VIA_NFKB
223,ICOSLG,hallmark,HALLMARK_INFLAMMATORY_RESPONSE
270,ICOSLG,hallmark,HALLMARK_ALLOGRAFT_REJECTION
398,FOSL2,hallmark,HALLMARK_HYPOXIA
...,...,...,...
878342,FOXO1,hallmark,HALLMARK_PANCREAS_BETA_CELLS
878418,GCG,hallmark,HALLMARK_PANCREAS_BETA_CELLS
878512,PDX1,hallmark,HALLMARK_PANCREAS_BETA_CELLS
878605,INS,hallmark,HALLMARK_PANCREAS_BETA_CELLS


In [ ]:
dc.run_ora(
    mat=adata,
    net=msigdb,
    source='geneset',
    target='genesymbol',
    verbose=True
)